<a href="https://colab.research.google.com/github/moeenessa31-lgtm/Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- In the first cell, we will import the libraries used in the project.

In [6]:
import pandas as pd
import numpy as np
import os

- Download data directly from Kaggle, via **API Token**

  -**(google.colab files)**
  This is the library used to upload API tokens

In [4]:
# upload API Token

from google.colab import files
files.upload()

# Create a new folder (directory) named .kaggle in the user's home folder (~)
!mkdir -p ~/.kaggle
# Copy the access key file to the folder we created in step one
!cp kaggle.json ~/.kaggle/
# Change the access permissions for the copied file to make it secure and private
!chmod 600 ~/.kaggle/kaggle.json

# Retrieve dataset from Kaggle command
!kaggle datasets download -d psparks/instacart-market-basket-analysis

# Unzip files command
!unzip -q instacart-market-basket-analysis.zip -d instacart_data

Saving kaggle.json to kaggle (3).json
Dataset URL: https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis
License(s): CC0-1.0
instacart-market-basket-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)
replace instacart_data/aisles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

- Test Example

In [7]:
# listdir() >> Available files
data = 'instacart_data/'
print("Available files:")
print(os.listdir(data))

print("-------------------------------")
ds = pd.read_csv(data + 'orders.csv')
ds.head()

Available files:
['order_products__prior.csv', 'order_products__train.csv', 'departments.csv', 'orders.csv', 'aisles.csv', 'products.csv']
-------------------------------


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0




---


# ***Data ingestion & memory-savvy joins, using python (not manual)! Your dataset includes multiple files***



---





In [8]:
# Understanding the original datatypes

aisles = pd.read_csv(data + 'aisles.csv')
print(aisles.dtypes)

print("---------------------------------------")

departments = pd.read_csv(data + 'departments.csv')
print(departments.dtypes)

print("---------------------------------------")

products = pd.read_csv(data + 'products.csv')
print(products.dtypes)

print("---------------------------------------")

orders = pd.read_csv(data + 'orders.csv')
print(orders.dtypes)

print("---------------------------------------")

order_products_prior = pd.read_csv(data + 'order_products__prior.csv')
print(order_products_prior.dtypes)

aisle_id     int64
aisle       object
dtype: object
---------------------------------------
department_id     int64
department       object
dtype: object
---------------------------------------
product_id        int64
product_name     object
aisle_id          int64
department_id     int64
dtype: object
---------------------------------------
order_id                    int64
user_id                     int64
eval_set                   object
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object
---------------------------------------
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtype: object


In [9]:
# Defining data types to reduce memory
Types = {
    'order_id': 'uint32',
    'user_id': 'uint32',
    'product_id': 'UInt16',
    'add_to_cart_order': 'UInt8',
    'reordered': 'UInt8',
    'aisle_id': 'UInt8',
    'department_id': 'UInt8',
    'order_dow': 'UInt8',
    'order_hour_of_day': 'UInt8',
    'days_since_prior_order': 'float16',
    'order_number': 'UInt8'
}

In [10]:
# Reading files using enhanced data types
aisles = pd.read_csv(data + 'aisles.csv', dtype={i: Types.get(i, None) for i in ['aisle_id']})

departments = pd.read_csv(data + 'departments.csv', dtype={i: Types.get(i, None) for i in ['department_id']})

products = pd.read_csv(data + 'products.csv', dtype={i: Types.get(i, None) for i in ['product_id', 'aisle_id', 'department_id']})

# Read orders, excluding 'days_since_prior_order' from explicit dtype setting during read_csv
orders = pd.read_csv(data + 'orders.csv', dtype={i: Types.get(i, None) for i in ['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day']})
# Convert 'days_since_prior_order' to float16 after loading the DataFrame
orders['days_since_prior_order'] = orders['days_since_prior_order'].astype('float16')

order_products_prior = pd.read_csv(data + 'order_products__prior.csv', dtype={i: Types.get(i, None) for i in ['order_id', 'product_id', 'add_to_cart_order', 'reordered']})

In [11]:
print(aisles.dtypes)

print("---------------------------------------")

print(departments.dtypes)

print("---------------------------------------")

print(products.dtypes)

print("---------------------------------------")

print(orders.dtypes)

print("---------------------------------------")

print(order_products_prior.dtypes)

aisle_id     UInt8
aisle       object
dtype: object
---------------------------------------
department_id     UInt8
department       object
dtype: object
---------------------------------------
product_id       UInt16
product_name     object
aisle_id          UInt8
department_id     UInt8
dtype: object
---------------------------------------
order_id                   uint32
user_id                    uint32
eval_set                   object
order_number                UInt8
order_dow                   UInt8
order_hour_of_day           UInt8
days_since_prior_order    float16
dtype: object
---------------------------------------
order_id             uint32
product_id           UInt16
add_to_cart_order     UInt8
reordered             UInt8
dtype: object





---



* **Merge files to obtain the required dataset**  


---




In [12]:
data_set = pd.merge(orders, order_products_prior, on='order_id', how='left')
data_set = pd.merge(data_set, products, on='product_id', how='left')
data_set = pd.merge(data_set, aisles, on='aisle_id', how='left')
data_set = pd.merge(data_set, departments, on='department_id', how='left')

In [13]:
data_set.head()

/usr/local/lib/python3.12/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.12/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,soft drinks,beverages
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,soy lactosefree,dairy eggs
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,popcorn jerky,snacks
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,popcorn jerky,snacks
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,paper goods,household


In [14]:
print(data_set.shape)
print("---------------------")
data_set.info()

(32640698, 15)
---------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32640698 entries, 0 to 32640697
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                uint32 
 1   user_id                 uint32 
 2   eval_set                object 
 3   order_number            UInt8  
 4   order_dow               UInt8  
 5   order_hour_of_day       UInt8  
 6   days_since_prior_order  float16
 7   product_id              UInt16 
 8   add_to_cart_order       UInt8  
 9   reordered               UInt8  
 10  product_name            object 
 11  aisle_id                UInt8  
 12  department_id           UInt8  
 13  aisle                   object 
 14  department              object 
dtypes: UInt16(1), UInt8(7), float16(1), object(4), uint32(2)
memory usage: 1.8+ GB




---


# ***- Exploratory Data Analysis***



---



## Missing Values
The current task focuses on exploring different strategies for handling missing values in the `data_set` DataFrame and evaluating their impact. The plan will be executed as follows:

1.  **Drop Rows with Missing Product Information**: Create a copy of the `data_set` and remove rows where critical product-related columns (`product_id`, `product_name`, `aisle_id`, `department_id`, `aisle`, `department`) are missing. These missing values are primarily associated with `eval_set` values 'test' and 'train', which do not have product details in the `order_products__prior.csv` file.
2.  **Impute 'days_since_prior_order' with Median**: Create another copy of the `data_set` and fill the missing values in `days_since_prior_order` with its median.
3.  **Impute Categorical Columns with Mode**: Create a third copy of the `data_set` and impute missing values in categorical columns (`product_name`, `aisle`, `department`) with their respective modes.
4.  **Visualize Impact**: Generate visualizations (e.g., distributions) to compare the original data with the datasets after applying each strategy.
5.  **Evaluate and Select Best Strategy**: Analyze the effects of each strategy on data integrity and suitability, and then recommend the most appropriate one.

Let's begin by implementing the first strategy: dropping rows with missing product information.

## Strategy 1: Drop Rows with Missing Values

### Subtask:
Create a copy of the dataset and apply a strategy to drop rows where critical columns (e.g., 'product_id', 'product_name') have missing values. This will likely focus on the 'test' and 'train' eval_set orders that do not have product_id.


**Reasoning**:
First, I need to create a copy of the original dataset to avoid modifying it directly for this strategy. Then, I will store the initial number of rows in the copied dataset before proceeding with the dropping operation as instructed.



In [15]:
data_set_strategy1 = data_set.copy()
initial_rows = data_set_strategy1.shape[0]
print(f"Initial number of rows in data_set_strategy1: {initial_rows}")

Initial number of rows in data_set_strategy1: 32640698


**Reasoning**:
Now that a copy of the dataframe has been created and initial rows captured, I will proceed with dropping rows where 'product_id' is null, then calculate and display the new row count, the number of rows dropped, the remaining missing values, and the 'eval_set' distribution, as per the instructions.



In [16]:
data_set_strategy1.dropna(subset=['product_id'], inplace=True)
rows_after_dropping = data_set_strategy1.shape[0]

print(f"Number of rows after dropping missing product_id: {rows_after_dropping}")
print(f"Number of rows dropped: {initial_rows - rows_after_dropping}")
print("\nMissing values after Strategy 1 (dropping rows with missing product_id):")

# Calculate and display remaining missing values for data_set_strategy1
miss_value_strategy1 = data_set_strategy1.isnull().sum()
miss_per_strategy1 = (miss_value_strategy1 / len(data_set_strategy1)) * 100
set_missing_strategy1 = pd.DataFrame({'Missing Count': miss_value_strategy1, 'Missing Percentage': miss_per_strategy1})
print(set_missing_strategy1[set_missing_strategy1['Missing Count'] > 0])

print("\n'eval_set' distribution after Strategy 1:")
display(data_set_strategy1['eval_set'].value_counts())

Number of rows after dropping missing product_id: 32434489
Number of rows dropped: 206209

Missing values after Strategy 1 (dropping rows with missing product_id):
                        Missing Count  Missing Percentage
days_since_prior_order        2078068             6.40697

'eval_set' distribution after Strategy 1:


,count
eval_set,
prior,32434489


## Strategy 2: Impute 'days_since_prior_order' (Median)

### Subtask:
Create another copy of the dataset and impute the missing values in 'days_since_prior_order' with its median value.


**Reasoning**:
To implement Strategy 2, I need to create a copy of the original dataset, calculate the median of 'days_since_prior_order', fill the missing values with this median, confirm no rows were dropped, and then display the remaining missing values and the 'eval_set' distribution.



In [17]:
data_set_strategy2 = data_set.copy()

initial_rows_strategy2 = data_set_strategy2.shape[0]
print(f"Initial number of rows in data_set_strategy2: {initial_rows_strategy2}")

# Calculate the median of 'days_since_prior_order'
median_days_since_prior_order = data_set_strategy2['days_since_prior_order'].median()
print(f"Median of 'days_since_prior_order': {median_days_since_prior_order}")

# Fill missing values in 'days_since_prior_order' with the median
data_set_strategy2['days_since_prior_order'].fillna(median_days_since_prior_order, inplace=True)

rows_after_imputation_strategy2 = data_set_strategy2.shape[0]
print(f"Number of rows after imputation: {rows_after_imputation_strategy2}")
print(f"Number of rows dropped (should be 0): {initial_rows_strategy2 - rows_after_imputation_strategy2}")

print("\nMissing values after Strategy 2 (imputing 'days_since_prior_order'):")
# Calculate and display remaining missing values for data_set_strategy2
miss_value_strategy2 = data_set_strategy2.isnull().sum()
miss_per_strategy2 = (miss_value_strategy2 / len(data_set_strategy2)) * 100
set_missing_strategy2 = pd.DataFrame({'Missing Count': miss_value_strategy2, 'Missing Percentage': miss_per_strategy2})
print(set_missing_strategy2[set_missing_strategy2['Missing Count'] > 0])

print("\n'eval_set' distribution after Strategy 2:")
display(data_set_strategy2['eval_set'].value_counts())

Initial number of rows in data_set_strategy2: 32640698
Median of 'days_since_prior_order': 8.0
Number of rows after imputation: 32640698
Number of rows dropped (should be 0): 0

Missing values after Strategy 2 (imputing 'days_since_prior_order'):


/tmp/ipython-input-1963363157.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_set_strategy2['days_since_prior_order'].fillna(median_days_since_prior_order, inplace=True)


                   Missing Count  Missing Percentage
product_id                206209            0.631754
add_to_cart_order         206209            0.631754
reordered                 206209            0.631754
product_name              206209            0.631754
aisle_id                  206209            0.631754
department_id             206209            0.631754
aisle                     206209            0.631754
department                206209            0.631754

'eval_set' distribution after Strategy 2:


,count
eval_set,
prior,32434489
train,131209
test,75000


## Strategy 3: Impute Categorical Columns with Mode

### Subtask:
Create a third copy of the `data_set` and impute missing values in categorical columns (`product_name`, `aisle`, `department`) with their respective modes.

**Reasoning**:
To implement Strategy 3, I will create a copy of the original dataset, calculate the mode for each specified categorical column, and then use these modes to fill the missing values. Afterwards, I will verify the integrity of the dataset by checking for dropped rows and re-evaluating missing values and the 'eval_set' distribution.

In [18]:
data_set_strategy3 = data_set.copy()

initial_rows_strategy3 = data_set_strategy3.shape[0]
print(f"Initial number of rows in data_set_strategy3: {initial_rows_strategy3}")

categorical_cols_to_impute = ['product_name', 'aisle', 'department']

for col in categorical_cols_to_impute:
    mode_value = data_set_strategy3[col].mode()[0]
    data_set_strategy3[col] = data_set_strategy3[col].fillna(mode_value)
    print(f"Mode for '{col}': {mode_value}")

rows_after_imputation_strategy3 = data_set_strategy3.shape[0]
print(f"Number of rows after imputation: {rows_after_imputation_strategy3}")
print(f"Number of rows dropped: {initial_rows_strategy3 - rows_after_imputation_strategy3}")

print("\nMissing values after Strategy 3 (imputing categorical columns with mode):")
# Calculate and display remaining missing values for data_set_strategy3
miss_value_strategy3 = data_set_strategy3.isnull().sum()
miss_per_strategy3 = (miss_value_strategy3 / len(data_set_strategy3)) * 100
set_missing_strategy3 = pd.DataFrame({'Missing Count': miss_value_strategy3, 'Missing Percentage': miss_per_strategy3})
print(set_missing_strategy3[set_missing_strategy3['Missing Count'] > 0])

print("\n'eval_set' distribution after Strategy 3:")
display(data_set_strategy3['eval_set'].value_counts())

Initial number of rows in data_set_strategy3: 32640698
Mode for 'product_name': Banana
Mode for 'aisle': fresh fruits
Mode for 'department': produce
Number of rows after imputation: 32640698
Number of rows dropped: 0

Missing values after Strategy 3 (imputing categorical columns with mode):
                        Missing Count  Missing Percentage
days_since_prior_order        2078068            6.366494
product_id                     206209            0.631754
add_to_cart_order              206209            0.631754
reordered                      206209            0.631754
aisle_id                       206209            0.631754
department_id                  206209            0.631754

'eval_set' distribution after Strategy 3:


,count
eval_set,
prior,32434489
train,131209
test,75000


## Visualize Impact of Strategies

### Subtask:
Generate visualizations to compare the distribution of key columns (days_since_prior_order, eval_set, and a categorical column imputed in Strategy 3) across the original dataset and the datasets after applying each missing value handling strategy.


**Reasoning**:
I need to import the `matplotlib.pyplot` and `seaborn` libraries to create the required visualizations. These libraries are essential for plotting histograms, KDEs, and count plots as specified in the instructions.



In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries matplotlib.pyplot and seaborn imported successfully.")

Libraries matplotlib.pyplot and seaborn imported successfully.


**Reasoning**:
I need to visualize the distribution of 'days_since_prior_order' across the original dataset and the datasets after applying each missing value handling strategy. I will use KDE plots to compare the distributions effectively, handling NaN values by dropping them for visualization purposes.



In [ ]:
plt.figure(figsize=(15, 10))
plt.suptitle('Distribution of Days Since Prior Order Across Strategies', fontsize=16)

# Original Data
plt.subplot(2, 2, 1)
sns.kdeplot(data_set['days_since_prior_order'].dropna(), fill=True)
plt.title('Original Data')
plt.xlabel('Days Since Prior Order')
plt.ylabel('Density')

# Strategy 1: Rows Dropped
plt.subplot(2, 2, 2)
sns.kdeplot(data_set_strategy1['days_since_prior_order'].dropna(), fill=True)
plt.title('Strategy 1: Rows Dropped (Product ID NaN)')
plt.xlabel('Days Since Prior Order')
plt.ylabel('Density')

# Strategy 2: Median Imputation
plt.subplot(2, 2, 3)
sns.kdeplot(data_set_strategy2['days_since_prior_order'], fill=True)
plt.title('Strategy 2: Median Imputation')
plt.xlabel('Days Since Prior Order')
plt.ylabel('Density')

# Strategy 3: Mode Imputation (Categorical)
plt.subplot(2, 2, 4)
sns.kdeplot(data_set_strategy3['days_since_prior_order'].dropna(), fill=True)
plt.title('Strategy 3: Mode Imputation (Categorical)')
plt.xlabel('Days Since Prior Order')
plt.ylabel('Density')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
